In [ ]:
import os
import matplotlib.pyplot as plt
import tensorflow as tf

In [3]:
## 1. Dataset das imagens
dataset_dir = os.path.join(os.getcwd(), 'imagens')
dataset_morango_dir = os.path.join(dataset_dir, 'morango')
dataset_pessego_dir = os.path.join(dataset_dir, 'pessego')
dataset_roma_dir = os.path.join(dataset_dir, 'roma')

dataset_morango_len = len(os.listdir(dataset_morango_dir))
dataset_pessego_len = len(os.listdir(dataset_pessego_dir))
dataset_roma_len = len(os.listdir(dataset_roma_dir))

print(f'Contagem de imagens de morango: {dataset_morango_len}')
print(f'Contagem de imagens de pêssego: {dataset_pessego_len}')
print(f'Contagem de imagens de romã: {dataset_roma_len}')

Contagem de imagens de morango: 306
Contagem de imagens de pêssego: 304
Contagem de imagens de romã: 311


In [4]:
# 2. Aplicando o rebalanceamento do dataset
# -> Utilizada a técnica de 'data augmentation' para gerar novas imagens de morango e pessego
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Criar um objeto ImageDataGenerator para data augmentation
datagen = ImageDataGenerator(
    rotation_range=20,          # Faixa de rotação aleatória em graus
    width_shift_range=0.2,      # Faixa de deslocamento horizontal aleatório (como uma fração da largura total)
    height_shift_range=0.2,     # Faixa de deslocamento vertical aleatório (como uma fração da altura total)
    shear_range=0.2,            # Faixa de cisalhamento aleatório (em radianos)
    zoom_range=0.2,             # Faixa de zoom aleatório
    horizontal_flip=True,      # Inverter aleatoriamente as imagens horizontalmente (espelhamento)
    fill_mode='nearest'        # Estratégia de preenchimento usada para preencher novos pixels gerados após a rotação ou deslocamento
)

if (dataset_morango_len == 250) or (dataset_pessego_len == 250): 
    numero_imagens_aumentadas = 61
    classes_aumentadas = ['morango', 'pessego']

    for classe in classes_aumentadas:
        # Diretório da classe atual
        diretorio_classe = os.path.join(dataset_dir, classe)
        
        # Criar diretório para a classe aumentada, se não existir
        diretorio_destino_classe = os.path.join(dataset_dir, classe)
        os.makedirs(diretorio_destino_classe, exist_ok=True)
        
        # Lista de arquivos de imagem na classe atual
        imagens_classe = os.listdir(diretorio_classe)
        
        # Selecionar aleatoriamente algumas imagens existentes para data augmentation
        indices_amostra = np.random.choice(len(imagens_classe), numero_imagens_aumentadas, replace=True)
        
        # Para cada imagem de amostra selecionada
        for indice in indices_amostra:
            imagem_nome = imagens_classe[indice]
            imagem_path = os.path.join(diretorio_classe, imagem_nome)
            
            # Carregar imagem
            imagem = Image.open(imagem_path)
            imagem_array = np.array(imagem)
            imagem_array = imagem_array.reshape((1,) + imagem_array.shape)  # Reshape para (1, altura, largura, canais)
            
            # Gerar imagens aumentadas e salvar no diretório de destino
            for i, batch in enumerate(datagen.flow(imagem_array, batch_size=1)):
                if i >= 1:  # Quantidade de imagens aumentadas a serem geradas por imagem de amostra
                    break
                imagem_aumentada = batch[0].astype(np.uint8)  # Converter de volta para o formato de imagem
                nova_imagem_nome = f"{os.path.splitext(imagem_nome)[0]}_aug_{i}.jpg"
                nova_imagem_path = os.path.join(diretorio_destino_classe, nova_imagem_nome)
                nova_imagem = Image.fromarray(imagem_aumentada)
                nova_imagem.save(nova_imagem_path)

In [5]:
# 3. Definição e separação dos dados de treinamento e dados de teste
from sklearn.model_selection import train_test_split
import shutil

projeto_dir = os.getcwd() # Diretório do projeto
dataset_treinamento_dir = os.path.join(projeto_dir, 'imagens_treinamento') # Caminho para imagens_treinamento
dataset_teste_dir = os.path.join(projeto_dir, 'imagens_teste') # Caminho para imagens_teste

proporcao_dataset_treinamento = 0.8 # 80% será utilizado no treinamento
proporcao_dataset_teste = 1 - proporcao_dataset_treinamento # 20% será utilizado no teste
random_seed = 42 # A resposta para a Vida - segundo livro de Aurélien Géron - gerar com random

classes = ['morango', 'pessego', 'roma']

# Função que executará a separação das imagens (utiliza algoritmo de aleatoriedade)
def split_dataset_to_train_and_test(classe):
    dataset_treinamento_classe_dir = os.path.join(dataset_treinamento_dir, classe) # Dir. destino treinamento classe
    dataset_teste_classe_dir = os.path.join(dataset_teste_dir, classe) # Dir. destino teste classe

    dataset_classe_dir = os.path.join(projeto_dir, 'imagens', classe) # Dir. origem imagens classe
    imagens_classe = [os.path.join(dataset_classe_dir, img) for img in os.listdir(dataset_classe_dir)] # Popular lista com imagens

    imagens_treinamento_classe, imagens_teste_classe = train_test_split(imagens_classe, test_size=proporcao_dataset_teste, random_state=random_seed) # Algoritmo que separa aleatoriamente as imagens de treinamento e de teste

    # Criação dos diretórios de treinamento e teste para a classe
    os.makedirs(dataset_treinamento_classe_dir, exist_ok=True)
    os.makedirs(dataset_teste_classe_dir, exist_ok=True)

    # Copia as imagens de treinamento para a pasta de treinamento da classe em questão
    for imagem in imagens_treinamento_classe:
        shutil.copy(imagem, dataset_treinamento_classe_dir) 

    # Copia as imagens de teste para a pasta de teste da classe em questão
    for imagem in imagens_teste_classe:
        shutil.copy(imagem, dataset_teste_classe_dir) 

# Verifica se os diretórios de treinamento e teste já existem
if not os.path.exists(dataset_treinamento_dir) or not os.path.exists(dataset_teste_dir):
    # Cria os diretórios de treinamento e teste
    os.makedirs(dataset_treinamento_dir, exist_ok=True)
    os.makedirs(dataset_teste_dir, exist_ok=True)
    # Para cada classe, executa a função de divisão de dados
    for classe in classes:
        split_dataset_to_train_and_test(classe)

In [6]:
# Como ficou dataset de treinamento e de testes:

dataset_treinamento_morango_len = len(os.listdir(os.path.join(dataset_treinamento_dir, 'morango')))
dataset_teste_morango_len = len(os.listdir(os.path.join(dataset_teste_dir, 'morango')))
dataset_treinamento_pessego_len = len(os.listdir(os.path.join(dataset_treinamento_dir, 'pessego')))
dataset_teste_pessego_len = len(os.listdir(os.path.join(dataset_teste_dir, 'pessego')))
dataset_treinamento_roma_len = len(os.listdir(os.path.join(dataset_treinamento_dir, 'roma')))
dataset_teste_roma_len = len(os.listdir(os.path.join(dataset_teste_dir, 'roma')))

print(f'Contagem de imagens de morango para treinamento: {dataset_treinamento_morango_len}')
print(f'Contagem de imagens de morango para teste: {dataset_teste_morango_len}')
print(f'Contagem de imagens de pêssego para treinamento: {dataset_treinamento_pessego_len}')
print(f'Contagem de imagens de pêssego para teste: {dataset_teste_pessego_len}')
print(f'Contagem de imagens de romã para treinamento: {dataset_treinamento_roma_len}')
print(f'Contagem de imagens de romã para teste: {dataset_teste_roma_len}')


Contagem de imagens de morango para treinamento: 244
Contagem de imagens de morango para teste: 62
Contagem de imagens de pêssego para treinamento: 243
Contagem de imagens de pêssego para teste: 61
Contagem de imagens de romã para treinamento: 248
Contagem de imagens de romã para teste: 63


In [12]:
## 4. Pré-processamento das imagens
## --> Definir tamanho de entrada das minhas imagens (em px)
## --> Definir qual estratégia de conversão adotar (scaling da imagem / foco no centro da imagem ignorando periferia / recortar imagem até no limite do tamanho definido e ignorar o restante)

'''
Considerações sobre a abordagem:
O pré-processamento é feito pela função "tf.keras.preprocessing.image_dataset_from_directory". Essa função permite especificar o tamanho das imagens e aplica automaticamente a normalização durante o carregamento das imagens. Além disso, aplica embaralhamento
e carrega as imagens em lotes (batch). Por fim, as imagens são consideradas como tensores do TensorFlow.
'''

import os
import tensorflow as tf

image_width = 300
image_heigth = 300
image_size = (image_width, image_heigth)

image_color_channel = 3
image_color_channel_size = 255
image_shape = image_size + (image_color_channel,)

batch_size = 32 # Valor que vou puxar do dataset por vez
epoch = 20 # Quantidade de vezes que vou percorrer meu dataset inteiro
learning_rate = 0.0001 # Taxa de aprendizagem

classes = ['morango', 'pessego', 'roma']

projeto_dir = os.getcwd()
dataset_treinamento_dir = os.path.join(projeto_dir, 'imagens_treinamento')
dataset_teste_dir = os.path.join(projeto_dir, 'imagens_teste')

print(dataset_treinamento_dir)
print(dataset_teste_dir)

data_set_treinamento = tf.keras.preprocessing.image_dataset_from_directory(
    dataset_treinamento_dir,
    image_size = image_size,
    batch_size = batch_size,
    shuffle = True, # Embaralhamento
    label_mode='categorical' # Carrega os dados em formato one-hot
)

data_set_teste = tf.keras.preprocessing.image_dataset_from_directory(
    dataset_teste_dir,
    image_size = image_size,
    batch_size = batch_size,
    shuffle = False, # Não é necessário embaralhar os dados de teste
    label_mode='categorical'
)

/home/matheus/Documentos/GitHub/esw-pin3-projeto/modelo_1/imagens_treinamento
/home/matheus/Documentos/GitHub/esw-pin3-projeto/modelo_1/imagens_teste
Found 735 files belonging to 3 classes.


Found 186 files belonging to 3 classes.


In [13]:
# 5. Definição da arquitetura da rede neural do modelo

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization, Input
from tensorflow.keras.losses import CategoricalCrossentropy
from tensorflow.keras.optimizers import Adam, SGD, RMSprop


# Arquitetura da rede do modelo implementado:
model = Sequential([
  # Definição do tipo de entrada: imagens de 300x300 pixels com 3 canais de cores (RGB) 
  Input(shape=(300,300,3)),

  # Primeira camada: convolucional com 32 filtros de tamanho 3x3, utilizando a função de ativação ReLU
  Conv2D(32, (3,3), activation='relu'),

  # Camada de normalização de batch, para normalizar a ativação da camada anterior
  BatchNormalization(),

  # Camada de MaxPooling, detalhada no documento de especificação do projeto
  MaxPooling2D((2,2)),

  # Mais uma camada Conv2D, com 64 filtros de tamanho 3x3, com função de ativação ReLU
  Conv2D(64, (3,3), activation='relu'),

  # Outra camada de normalização de batch, para normalizar a ativação da camada anterior
  BatchNormalization(),

  # Camada de MaxPooling, detalhada no documento de especificação do projeto
  MaxPooling2D((2,2)),

  # Camada de achatamento (Flatten) para transformar os mapas de características 2D em um vetor 1D
  Flatten(),
  
  # Camada densa (totalmente conectada) com 128 neurônios e função de ativação ReLU
  Dense(128, activation='relu'),
  
  # Mias uma camada de normalização de batch, para normalizar a ativação da camada anterior
  BatchNormalization(),

  # Mais uma camada densa com 64 neurônios e função de ativação ReLU
  Dense(64, activation='relu'),
  
  # Camada de Dropout para prevenir overfitting, desativando aleatoriamente 50% dos neurônios
  Dropout(0.5),
  
  # Camada de saída com 3 neurônios (um para cada classe) e função de ativação softmax para a classificação multiclasse
  Dense(3, activation='softmax')
])

# Compilação do modelo
# Aqui, utilizamos a função de perda conforme espeficicado no documento do projeto
# Função de perda: crossentropy, que irá calcular a diferença entre as previsões realizadas pelo modelo e os rótulos verdadeiros associados aos dados de treinamento.

# Além disso, testaremos 3 tipos de funções de otimização:

# Função de otimização Adam
model.compile(optimizer=Adam(learning_rate), loss=CategoricalCrossentropy(), metrics=['accuracy'])

# Função de otimização SGD
# model.compile(optimizer=SGD(), loss=CategoricalCrossentropy(), metrics=['accuracy'])

# Função de otimização RMSprop
# model.compile(optimizer=RMSprop(), loss=CategoricalCrossentropy(), metrics=['accuracy'])

In [16]:
# 6. Treinamento do modelo

# É passado o dataset de treinamento, a quantidade de épocas (quantas vezes vai percorrer o dataset) e o dataset de validação
model.fit(
  data_set_treinamento,
  epochs=epoch,
  validation_data=data_set_teste
)

# Avaliação do modelo
teste_perca, teste_acuracia = model.evaluate(data_set_teste)
print('Teste acurácia: ', teste_acuracia)

'''
Epochs: Cada linha corresponde a uma época de treinamento. Uma época é uma iteração completa sobre todo o conjunto de dados de treinamento.

Batches por Época: Aqui, o número 23/23 indica que há 23 batches (ou lotes) de dados sendo processados em cada época. Isso pode variar dependendo do tamanho do conjunto de treinamento e do tamanho do lote (batch_size). Por exemplo, se você tem 700 amostras de treinamento e está usando um tamanho de lote de 32, então cada época terá 22 batches (700/32 = 21.875).

Tempo por Época: O tempo indicado representa o tempo gasto para completar uma época. Isso pode variar dependendo da complexidade do modelo, tamanho do conjunto de dados e recursos do hardware.

Accuracy (Acurácia) e Loss (Perda): A acurácia e a perda durante o treinamento e validação são mostradas para cada época. A acurácia indica a proporção de previsões corretas em relação ao total de exemplos, enquanto a perda (loss) é uma medida do quão bem o modelo está performando durante o treinamento, sendo minimizada ao longo das épocas.

Val_accuracy e Val_loss: Estes são os valores de acurácia e perda calculados no conjunto de dados de validação, que é usado para avaliar o desempenho do modelo em dados que não foram vistos durante o treinamento. É importante monitorar essas métricas para evitar overfitting e garantir que o modelo esteja generalizando bem para novos dados.

Test accuracy e Test loss: Após o treinamento, o modelo é avaliado no conjunto de dados de teste (geralmente separado do conjunto de validação). Estas métricas fornecem uma avaliação final do desempenho do modelo em dados completamente novos e não vistos durante o treinamento.

Acurácia: No final do treinamento, a acurácia nos dados de validação foi de aproximadamente 84.41%. Isso significa que o modelo classificou corretamente cerca de 84.41% das amostras nos dados de validação.

Perda: A perda nos dados de validação foi de aproximadamente 0.4956. A perda é uma medida de quão boa ou ruim é a previsão do modelo para uma única amostra; valores mais baixos são melhores.

Tendência ao longo das épocas: A acurácia e a perda parecem estar se estabilizando ao longo das épocas, o que sugere que o modelo está convergindo para uma solução.
'''


Epoch 1/20
23/23 ━━━━━━━━━━━━━━━━━━━━ 137s 6s/step - accuracy: 0.5911 - loss: 1.2363 - val_accuracy: 0.4355 - val_loss: 8.8659
Epoch 2/20
23/23 ━━━━━━━━━━━━━━━━━━━━ 145s 6s/step - accuracy: 0.7868 - loss: 0.5642 - val_accuracy: 0.5914 - val_loss: 2.7070
Epoch 3/20
23/23 ━━━━━━━━━━━━━━━━━━━━ 121s 5s/step - accuracy: 0.8734 - loss: 0.3712 - val_accuracy: 0.5269 - val_loss: 1.4865
Epoch 4/20
23/23 ━━━━━━━━━━━━━━━━━━━━ 121s 5s/step - accuracy: 0.9033 - loss: 0.2868 - val_accuracy: 0.6183 - val_loss: 0.8187
Epoch 5/20
23/23 ━━━━━━━━━━━━━━━━━━━━ 121s 5s/step - accuracy: 0.9235 - loss: 0.2405 - val_accuracy: 0.8172 - val_loss: 0.4999
Epoch 6/20
23/23 ━━━━━━━━━━━━━━━━━━━━ 120s 5s/step - accuracy: 0.9510 - loss: 0.1896 - val_accuracy: 0.8333 - val_loss: 0.4396
Epoch 7/20
23/23 ━━━━━━━━━━━━━━━━━━━━ 128s 6s/step - accuracy: 0.9582 - loss: 0.1524 - val_accuracy: 0.8602 - val_loss: 0.4129
Epoch 8/20
23/23 ━━━━━━━━━━━━━━━━━━━━ 135s 6s/step - accuracy: 0.9662 - loss: 0.1233 - val_accuracy: 0.8763 - v